
<br>
Agglomerative clustering with and without structure<br>
===================================================<br>
This example shows the effect of imposing a connectivity graph to capture<br>
local structure in the data. The graph is simply the graph of 20 nearest<br>
neighbors.<br>
Two consequences of imposing a connectivity can be seen. First clustering<br>
with a connectivity matrix is much faster.<br>
Second, when using a connectivity matrix, single, average and complete<br>
linkage are unstable and tend to create a few clusters that grow very<br>
quickly. Indeed, average and complete linkage fight this percolation behavior<br>
by considering all the distances between two clusters when merging them (<br>
while single linkage exaggerates the behaviour by considering only the<br>
shortest distance between clusters). The connectivity graph breaks this<br>
mechanism for average and complete linkage, making them resemble the more<br>
brittle single linkage. This effect is more pronounced for very sparse graphs<br>
(try decreasing the number of neighbors in kneighbors_graph) and with<br>
complete linkage. In particular, having a very small number of neighbors in<br>
the graph, imposes a geometry that is close to that of single linkage,<br>
ich is well known to have this percolation instability. 
<br>
Authors: Gael Varoquaux, Nelle Varoquaux<br>
License: BSD 3 clause

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

Generate sample data

In [ ]:
n_samples = 1500
np.random.seed(0)
t = 1.5 * np.pi * (1 + 3 * np.random.rand(1, n_samples))
x = t * np.cos(t)
y = t * np.sin(t)

In [ ]:
X = np.concatenate((x, y))
X += .7 * np.random.randn(2, n_samples)
X = X.T

Create a graph capturing local connectivity. Larger number of neighbors<br>
will give more homogeneous clusters to the cost of computation<br>
time. A very large number of neighbors gives more evenly distributed<br>
cluster sizes, but may not impose the local manifold structure of<br>
the data

In [ ]:
knn_graph = kneighbors_graph(X, 30, include_self=False)

In [ ]:
for connectivity in (None, knn_graph):
    for n_clusters in (30, 3):
        plt.figure(figsize=(10, 4))
        for index, linkage in enumerate(('average',
                                         'complete',
                                         'ward',
                                         'single')):
            plt.subplot(1, 4, index + 1)
            model = AgglomerativeClustering(linkage=linkage,
                                            connectivity=connectivity,
                                            n_clusters=n_clusters)
            t0 = time.time()
            model.fit(X)
            elapsed_time = time.time() - t0
            plt.scatter(X[:, 0], X[:, 1], c=model.labels_,
                        cmap=plt.cm.nipy_spectral)
            plt.title('linkage=%s\n(time %.2fs)' % (linkage, elapsed_time),
                      fontdict=dict(verticalalignment='top'))
            plt.axis('equal')
            plt.axis('off')
            plt.subplots_adjust(bottom=0, top=.89, wspace=0,
                                left=0, right=1)
            plt.suptitle('n_cluster=%i, connectivity=%r' %
                         (n_clusters, connectivity is not None), size=17)

In [ ]:
plt.show()